In [27]:
def is_leap_year(year):
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True  # Divisible by 400, so it's a leap year
            else:
                return False  # Divisible by 100 but not 400, so it's not a leap year
        else:
            return True  # Divisible by 4 but not 100, so it's a leap year
    else:
        return False  # Not divisible by 4, so it's not a leap year

In [42]:
import pandas as pd
import os

work_dir = "/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/"
target_dir = '/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/'

rain_gaez_df = pd.read_pickle('/Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/pkl_files/rain_df.pkl')
temp_gaez_df = pd.read_pickle('/Users/kavya/Documents/ECS289L/Project2/GAEZ/tif_files/pkl_files/temp_df.pkl')

file_names = os.listdir(work_dir)

# Filter for only .tif files
met_files = [file for file in file_names if file.endswith('.met')]

for met_file_name in met_files:
    with open(file_path, "r") as file:
        lines = file.readlines()[:13]
        
    df = pd.read_csv(work_dir + met_file_name, delim_whitespace=True, skiprows=12)
    sector = int(met_file_name.split("-")[3].replace(".met", ""))

    # Assign new column names
    df.columns = ['year', 'day', 'radn', 'maxt', 'mint', 'rain']
    
    grouped = df.groupby('day')
    mean_values = grouped.mean()
    
    mean_values.reset_index(inplace=True)  # Reset the index
    mean_values = mean_values[df.columns]
    
    for year in range(2023,2100):
        if(is_leap_year(year)):
            temp_df = mean_values.copy()
            temp_df.loc[:, 'year'] = [year] * 366
        else:
            temp_df = mean_values[:365].copy()
            temp_df.loc[:, 'year'] = [year] * 365
        
        # rain mean shift
        rain_nasa_mean = temp_df['rain'].sum()

        rain_gaez_mean = rain_gaez_df[(rain_gaez_df['sector'] == sector) & (rain_gaez_df['year'] == str(year))]['rain']

        if(is_leap_year(year)):
            rain_diff = (rain_gaez_mean-rain_nasa_mean)/366

        else:
            rain_diff = (rain_gaez_mean-rain_nasa_mean)/365

        temp_df.loc[:, "rain"] = temp_df["rain"].apply(lambda x: x + rain_diff.values[0])

        # temp mean shift
        temp_nasa_mean = temp_df['maxt'].mean()

        temp_gaez_mean = temp_gaez_df[(temp_gaez_df['sector'] == sector) & (temp_gaez_df['year'] == str(year))]['temp']

        if(is_leap_year(year)):
            temp_diff = (temp_gaez_mean-temp_nasa_mean)/366

        else:
            temp_diff = (temp_gaez_mean-temp_nasa_mean)/365

        temp_df.loc[:, "maxt"] = temp_df["maxt"].apply(lambda x: x + rain_diff.values[0])
        df = pd.concat([df,temp_df])
    tar_file_path = target_dir+"2011-2100-sector-"+str(sector)+".met"
    print(tar_file_path)
    with open(tar_file_path, "w") as file:
        # Write the lines
        file.write(''.join(lines))
    
        # Write the DataFrame with a delimiter
        df.to_csv(file, sep=" ", index=False, header=False)

/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-41.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-40.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-42.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-43.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-2.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-18.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-30.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-24.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-25.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/interpolated/2011-2100-sector-31.met
/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/inte

In [39]:
file_path = "/Users/kavya/Documents/ECS289L/Project2/NASAPowerWeather/2011-2022-sector-1.met"
tar_file_path = "/Users/kavya/Documents/2011-2022-sector-1.met"

with open(file_path, "r") as file:
    lines = file.readlines()[:13]
    
df = pd.read_csv(file_path, delim_whitespace=True, skiprows=12)
df.columns = ['year', 'day', 'radn', 'maxt', 'mint', 'rain']
grouped = df.groupby('day')
mean_values = grouped.mean()
mean_values.loc[:, 'year'] = ['2071'] * 366
mean_values.reset_index(inplace=True)  # Reset the index
mean_values = mean_values[df.columns]
print(mean_values)
# Write the lines and DataFrame to the file
with open(tar_file_path, "w") as file:
    # Write the lines
    file.write(''.join(lines))
    
    # Write the DataFrame with a delimiter
    mean_values.to_csv(file, sep=" ", index=False, header=False)

     year  day      radn       maxt      mint      rain
0    2071    1  8.533333  13.075000  1.841667  1.183333
1    2071    2  7.500000  12.566667  2.750000  0.891667
2    2071    3  6.850000  13.633333  4.716667  3.433333
3    2071    4  7.341667  13.991667  5.316667  3.391667
4    2071    5  7.191667  14.208333  4.741667  4.383333
..    ...  ...       ...        ...       ...       ...
361  2071  362  8.400000  13.275000  2.775000  0.375000
362  2071  363  6.891667  12.625000  3.258333  3.333333
363  2071  364  7.933333  14.591667  3.275000  1.566667
364  2071  365  8.566667  14.033333  3.008333  4.466667
365  2071  366  9.200000  13.300000  1.533333  0.300000

[366 rows x 6 columns]


In [7]:
year = 2023

str(year)

'2023'